In [1]:
from google.colab import files 
uploaded = files.upload() 

Saving claimskg_result.csv to claimskg_result.csv


In [2]:
import io 
import pandas as pd
data = pd.read_csv(io.BytesIO(uploaded['claimskg_result.csv'])) 
print(type(data))
print(data.head())

<class 'pandas.core.frame.DataFrame'>
                                                  id  ... language
0  http://data.gesis.org/claimskg/claim_review/b0...  ...  English
1  http://data.gesis.org/claimskg/claim_review/03...  ...  English
2  http://data.gesis.org/claimskg/claim_review/e1...  ...  English
3  http://data.gesis.org/claimskg/claim_review/3c...  ...  English
4  http://data.gesis.org/claimskg/claim_review/b4...  ...  English

[5 rows x 14 columns]


In [11]:
data.columns   # after data.drop(["id","date","truthRating","author","headline","keywords","sourceURL","link","language"],axis=1,inplace=True)

Index(['text', 'ratingName', 'named_entities_claim', 'named_entities_article',
       'source'],
      dtype='object')

In [0]:
data.drop(["text"],axis=1,inplace=True)

In [13]:
data.columns

Index(['ratingName', 'named_entities_claim', 'named_entities_article',
       'source'],
      dtype='object')

In [14]:
data.shape

(10000, 4)

In [17]:
clean_data=data.dropna()
clean_data.head()

,ratingName,named_entities_claim,named_entities_article,source
0,False,"Netflix,Netflix and Chill,Republican,Ted Cruz,...","Netflix and Chill,Ted Cruz",snopes
2,False,"ABCNews.com,California Department of Public He...","California Department of Public Health,measles",snopes
3,False,"2008. Democratic presidential nominee,Alfred...","Michelle Obama,Waldorf-Astoria hotel,room service",snopes
4,True,"Associated Press,Bonkers,Facebook,Herendeen,Ne...",Silly Bandz,snopes
5,False,"Bay of Fundy,Brine Shrimp,Brine shrimp,Kittery...","brine shrimp,lobster,vagina",snopes


In [18]:
clean_data.shape

(6656, 4)

In [19]:
data_x=clean_data.iloc[:,1:]
data_x.head()

,named_entities_claim,named_entities_article,source
0,"Netflix,Netflix and Chill,Republican,Ted Cruz,...","Netflix and Chill,Ted Cruz",snopes
2,"ABCNews.com,California Department of Public He...","California Department of Public Health,measles",snopes
3,"2008. Democratic presidential nominee,Alfred...","Michelle Obama,Waldorf-Astoria hotel,room service",snopes
4,"Associated Press,Bonkers,Facebook,Herendeen,Ne...",Silly Bandz,snopes
5,"Bay of Fundy,Brine Shrimp,Brine shrimp,Kittery...","brine shrimp,lobster,vagina",snopes


In [20]:
data_y=clean_data.iloc[:,:1]
data_y.head()

,ratingName
0,False
2,False
3,False
4,True
5,False


In [21]:
print(data_x.shape)
print(data_y.shape)

(6656, 3)
(6656, 1)


In [0]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
#from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from matplotlib import pyplot as plt

In [0]:
def split_data(x,y):
    
    """
    Reminder : Clean the set before sending here.
    Input  : x=Attributes of the dataset, y=Target Attribute
    Output : Split data of train test
    
    We are removing dev , as dev can be better used for the models that includes DNN, CNN and RNN
    
    """
    X=x
    Y=y
    
    train_x,test_x,train_y,test_y = train_test_split(x,y,test_size=0.2,random_state=12)
    
    #train_x : Training data with attributes
    #train_y : Training data with ratings
    #test_x  : testing data with attributes
    #test_y  : testing data with rating 
    
    print("Final size of train/test :",train_x.size,"/",test_x.size)
    return train_x,train_y,test_x,test_y

In [0]:
from sklearn.model_selection import GridSearchCV
def navie_bayes(train_x,train_y,test_x,test_y,itera=100):  #itera changed to 100
   
    print('model building')
    NBG = MultinomialNB()
    if itera == 100:
        alphas=np.arange(0.001,1,0.001)
    else:
        alphas=np.arange(0.01,1,0.01)
    print('begin grid')
    grid = GridSearchCV(estimator=NBG, param_grid=dict(alpha=alphas))

    grid.fit(train_x, train_y)
    
    #print(grid)
    # summarize the results of the grid search
    print("grid score",grid.best_score_)
    print("grid alpha",grid.best_estimator_.alpha)
    alpha=grid.best_estimator_.alpha
    
    NB = MultinomialNB(alpha = alpha )
    NB.fit(train_x,train_y)  
    predict_NB_train = NB.predict(train_x) 
    predict_NB_test = NB.predict(test_x) 
    score_train = metrics.accuracy_score(train_y, predict_NB_train) 
    score_test = metrics.accuracy_score(test_y, predict_NB_test)
      
    print("Alpha: {:.3f} test_Score: {:.5f} train_score: {:.5f}".format(alpha,score_test,score_train))# checking tha accuracy 
       
    #print("best accuracy with Alpha: {:.2f} Score: {:.5f}".format(alpha_value, best_score))
    cm_train= metrics.confusion_matrix(train_y, predict_NB_train)
    print("training confusion matrix")
    print(cm_train)
    cm_test = metrics.confusion_matrix(test_y, predict_NB_test)
    print("testing confusion matrix")
    print(cm_test)
    
    print("------------------------------------------")

In [25]:
import numpy as np
a=np.array([])
a.size

0

In [0]:
def mult_tfidf(x1,x2,y,x3=a):
    from sklearn.feature_extraction.text import TfidfVectorizer

    if x3.size==0:
        train_x1,train_y1,test_x1,test_y1=split_data(x1,y)
        train_x2,train_y2,test_x2,test_y2=split_data(x2,y)



        tfidf_vectorizer = TfidfVectorizer(stop_words='english',max_df=0.7,min_df=0.02)   # a TFIDF vectorizer
        tfidf_train_x1 = pd.DataFrame(tfidf_vectorizer.fit_transform(train_x1).todense())  #fitting the training data
        tfidf_test_x1 = pd.DataFrame(tfidf_vectorizer.transform(test_x1).todense())

        tfidf_train_x2 = pd.DataFrame(tfidf_vectorizer.fit_transform(train_x2).todense())  #fitting the training data
        tfidf_test_x2 = pd.DataFrame(tfidf_vectorizer.transform(test_x2).todense()) 

        train=pd.concat([tfidf_train_x1,tfidf_train_x2],axis=1)
        test=pd.concat([tfidf_test_x1,tfidf_test_x2],axis=1)
        print(train.shape)
        print(test.shape)
        print(train_y2.shape)
        print(test_y2.shape)

        navie_bayes(tfidf_train_x1,train_y1,tfidf_test_x1,test_y1)
    
    else:
        train_x1,train_y1,test_x1,test_y1=split_data(x1,y)
        train_x2,train_y2,test_x2,test_y2=split_data(x2,y)
        train_x3,train_y3,test_x3,test_y3=split_data(x3,y)
        
        tfidf_vectorizer = TfidfVectorizer(stop_words='english',max_df=0.7,min_df=0.02)   # a TFIDF vectorizer
        tfidf_train_x1 = pd.DataFrame(tfidf_vectorizer.fit_transform(train_x1).todense())  #fitting the training data
        tfidf_test_x1 = pd.DataFrame(tfidf_vectorizer.transform(test_x1).todense())

        tfidf_train_x2 = pd.DataFrame(tfidf_vectorizer.fit_transform(train_x2).todense())  #fitting the training data
        tfidf_test_x2 = pd.DataFrame(tfidf_vectorizer.transform(test_x2).todense()) 
        
        tfidf_train_x3 = pd.DataFrame(tfidf_vectorizer.fit_transform(train_x3).todense())  #fitting the training data
        tfidf_test_x3 = pd.DataFrame(tfidf_vectorizer.transform(test_x3).todense())
        
        train=pd.concat([tfidf_train_x1,tfidf_train_x2,tfidf_train_x3],axis=1)
        test=pd.concat([tfidf_test_x1,tfidf_test_x2,tfidf_test_x3],axis=1)
        
        navie_bayes(train,train_y2,test,test_y2)

In [27]:
mult_tfidf(x1=data_x.named_entities_claim,y=data_y.ratingName)  #single attr named_entities_claim

Final size of train/test : 5324 / 1332
(5324, 153)
(1332, 153)
model building
begin grid
grid score 0.7663410286279078
grid alpha 0.001
Alpha: 0.001 test_Score: 0.76952 train_score: 0.76709
training confusion matrix
[[4076    6]
 [1234    8]]
testing confusion matrix
[[1022    0]
 [ 307    3]]
------------------------------------------


In [28]:
mult_tfidf(x1=data_x.named_entities_article,y=data_y.ratingName) #single attr named_entities_article

Final size of train/test : 5324 / 1332
(5324, 7)
(1332, 7)
model building
begin grid
grid score 0.7667167919799498
grid alpha 0.001
Alpha: 0.001 test_Score: 0.76727 train_score: 0.76672
training confusion matrix
[[4082    0]
 [1242    0]]
testing confusion matrix
[[1022    0]
 [ 310    0]]
------------------------------------------


In [30]:
mult_tfidf(x1=data_x.source,y=data_y.ratingName)  #single attr source

Final size of train/test : 5324 / 1332
(5324, 4)
(1332, 4)
model building
begin grid
grid score 0.7667167919799498
grid alpha 0.001
Alpha: 0.001 test_Score: 0.76727 train_score: 0.76672
training confusion matrix
[[4082    0]
 [1242    0]]
testing confusion matrix
[[1022    0]
 [ 310    0]]
------------------------------------------


In [32]:
mult_tfidf(x1=data_x.named_entities_claim,x2=data_x.named_entities_article,y=data_y.ratingName,x3=a) #named_e_claim and named_e_article attr

Final size of train/test : 5324 / 1332
Final size of train/test : 5324 / 1332
(5324, 160)
(1332, 160)
(5324,)
(1332,)
model building
begin grid
grid score 0.7663410286279078
grid alpha 0.001
Alpha: 0.001 test_Score: 0.76952 train_score: 0.76709
training confusion matrix
[[4076    6]
 [1234    8]]
testing confusion matrix
[[1022    0]
 [ 307    3]]
------------------------------------------


In [33]:
mult_tfidf(x1=data_x.named_entities_claim,x2=data_x.source,y=data_y.ratingName,x3=a) #named_e_claim and source attr

Final size of train/test : 5324 / 1332
Final size of train/test : 5324 / 1332
(5324, 157)
(1332, 157)
(5324,)
(1332,)
model building
begin grid
grid score 0.7663410286279078
grid alpha 0.001
Alpha: 0.001 test_Score: 0.76952 train_score: 0.76709
training confusion matrix
[[4076    6]
 [1234    8]]
testing confusion matrix
[[1022    0]
 [ 307    3]]
------------------------------------------


In [34]:
mult_tfidf(x1=data_x.named_entities_article,x2=data_x.source,y=data_y.ratingName,x3=a)  # named_e_article and source attr

Final size of train/test : 5324 / 1332
Final size of train/test : 5324 / 1332
(5324, 11)
(1332, 11)
(5324,)
(1332,)
model building
begin grid
grid score 0.7667167919799498
grid alpha 0.001
Alpha: 0.001 test_Score: 0.76727 train_score: 0.76672
training confusion matrix
[[4082    0]
 [1242    0]]
testing confusion matrix
[[1022    0]
 [ 310    0]]
------------------------------------------


In [38]:
mult_tfidf(x1=data_x.named_entities_claim,x2=data_x.named_entities_article,y=data_y.ratingName,x3=data_x.source) #all attr

Final size of train/test : 5324 / 1332
Final size of train/test : 5324 / 1332
Final size of train/test : 5324 / 1332
model building
begin grid
grid score 0.7650259451445515
grid alpha 0.579
Alpha: 0.579 test_Score: 0.77177 train_score: 0.76634
training confusion matrix
[[4018   64]
 [1180   62]]
testing confusion matrix
[[1014    8]
 [ 296   14]]
------------------------------------------
